<a href="https://colab.research.google.com/github/aniket-alt/Autogluon/blob/main/autogluon_kaggle_California_House_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to use AutoGluon for Kaggle competitions

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/autogluon/autogluon/blob/stable/docs/tutorials/tabular/advanced/tabular-kaggle.ipynb)
[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/autogluon/autogluon/blob/stable/docs/tutorials/tabular/advanced/tabular-kaggle.ipynb)



This tutorial will teach you how to use AutoGluon to become a serious Kaggle competitor without writing lots of code.
We first outline the general steps to use AutoGluon in Kaggle contests. Here, we assume the competition involves tabular data which are stored in one (or more) CSV files.

1) Run Bash command: pip install kaggle

2) Navigate to: https://www.kaggle.com/account and create an account (if necessary).
Then , click on "Create New API Token" and move downloaded file to this location on your machine: `~/.kaggle/kaggle.json`. For troubleshooting, see [Kaggle API instructions](https://www.kaggle.com/docs/api).

3) To download data programmatically: Execute this Bash command in your terminal:

`kaggle competitions download -c [COMPETITION]`

Here, [COMPETITION] should be replaced by the name of the competition you wish to enter.
Alternatively, you can download data manually: Just navigate to website of the Kaggle competition you wish to enter, click "Download All", and accept the competition's terms.

4) If the competition's training data is comprised of multiple CSV files, use [pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) to properly merge/join them into a single data table where rows = training examples, columns = features.

5) Run autogluon `fit()` on the resulting data table.

6) Load the test dataset from competition (again making the necessary merges/joins to ensure it is in the exact same format as the training data table), and then call autogluon `predict()`.  Subsequently use [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) to load the competition's `sample_submission.csv` file into a DataFrame, put the AutoGluon predictions in the right column of this DataFrame, and finally save it as a CSV file via [pandas.to_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html). If the competition does not offer a sample submission file, you will need to create the submission file yourself by appropriately reformatting AutoGluon's test predictions.

7) Submit your predictions via Bash command:

`kaggle competitions submit -c [COMPETITION] -f [FILE] -m ["MESSAGE"]`

Here, [COMPETITION] again is the competition's name, [FILE] is the name of the CSV file you created with your predictions, and ["MESSAGE"] is a string message you want to record with this submitted entry. Alternatively, you can  manually upload your file of predictions on the competition website.

8) Finally, navigate to competition leaderboard website to see how well your submission performed!
It may take time for your submission to appear.



Below, we demonstrate how to do steps (4)-(6) in Python for a specific Kaggle competition: [ieee-fraud-detection](https://www.kaggle.com/c/ieee-fraud-detection/).
This means you'll need to run the above steps with `[COMPETITION]` replaced by `ieee-fraud-detection` in each command.  Here, we assume you've already completed steps (1)-(3) and the data CSV files are available on your computer. To begin step (4), we first load the competition's training data into Python:

In [1]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

!kaggle competitions download -c california-house-prices

!unzip /content/california-house-prices.zip -d /content/california-house-prices

  0% 0.00/29.5M [00:00<?, ?B/s]
100% 29.5M/29.5M [00:00<00:00, 1.60GB/s]
Archive:  /content/california-house-prices.zip
  inflating: /content/california-house-prices/sample_submission.csv  
  inflating: /content/california-house-prices/test.csv  
  inflating: /content/california-house-prices/train.csv  


In [2]:
!pip3 install -U pip
!pip3 install -U setuptools wheel

!pip3 install autogluon

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 73.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.7 MB/s  0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━

In [3]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

directory = './california-house-prices/'  # directory where you have downloaded the data CSV files from the competition
# The label column for this dataset is 'median_house_value'
label = 'Sold Price'
eval_metric = 'rmse'  # Optional: specify that competition evaluation metric is RMSE for regression tasks
save_path = directory + 'AutoGluonModels/'  # where to store trained models

train_data = pd.read_csv(directory+'train.csv')
test_data = pd.read_csv(directory+'test.csv')

predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path, verbosity=3).fit( train_data, presets='best_quality', time_limit=300)

results = predictor.fit_summary()

Now, we use the trained AutoGluon Predictor to make predictions on the competition's test data. It is imperative that multiple test data files are joined together in the exact same manner as the training data. Because this competition is evaluated based on the AUC (Area under the ROC curve) metric, we ask AutoGluon for predicted class-probabilities rather than class predictions. In general, when to use `predict` vs `predict_proba` will depend on the particular competition.

In [4]:
predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path, verbosity=3).fit(
    train_data, presets='medium_quality', time_limit=600)

results = predictor.fit_summary()

Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          12
GPU Count:          1
Memory Avail:       50.99 GB / 52.96 GB (96.3%)
Disk Space Avail:   189.98 GB / 235.68 GB (80.6%)
Presets specified: ['medium_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': False}
Full kwargs:
{'_experimental_dynamic_hyperparameters': False,
 '_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': False,
 'calibrate': 'auto',
 'delay_bag_sets': False,
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_callbacks': False,
             'enable_ray_logging': True,
             'holdout_data': None,
             'holdout_frac': 0.

[50]	valid_set's rmse: 500535
[100]	valid_set's rmse: 432890
[150]	valid_set's rmse: 410792
[200]	valid_set's rmse: 398594
[250]	valid_set's rmse: 395181
[300]	valid_set's rmse: 389588
[350]	valid_set's rmse: 392043


Saving /content/california-house-prices/AutoGluonModels/models/LightGBMXT/model.pkl
Saving /content/california-house-prices/AutoGluonModels/utils/attr/LightGBMXT/y_pred_proba_val.pkl
	-389418.2663	 = Validation score   (-root_mean_squared_error)
	27.57s	 = Training   runtime
	0.12s	 = Validation runtime
	20993.4	 = Inference  throughput (rows/s | 2500 batch size)
Saving /content/california-house-prices/AutoGluonModels/models/trainer.pkl
Fitting model: LightGBM ... Training model for up to 421.59s of the 421.58s of remaining time.
	Fitting LightGBM with 'num_gpus': 0, 'num_cpus': 6
	Fitting with cpus=6, gpus=0, mem=5.6/48.6 GB
	Fitting 10000 rounds... Hyperparameters: {'learning_rate': 0.05}


[50]	valid_set's rmse: 333904
[100]	valid_set's rmse: 362304


Saving /content/california-house-prices/AutoGluonModels/models/LightGBM/model.pkl
Saving /content/california-house-prices/AutoGluonModels/utils/attr/LightGBM/y_pred_proba_val.pkl
	-327695.8952	 = Validation score   (-root_mean_squared_error)
	20.74s	 = Training   runtime
	0.09s	 = Validation runtime
	26630.6	 = Inference  throughput (rows/s | 2500 batch size)
Saving /content/california-house-prices/AutoGluonModels/models/trainer.pkl
Fitting model: RandomForestMSE ... Training model for up to 400.74s of the 400.73s of remaining time.
	Fitting RandomForestMSE with 'num_gpus': 0, 'num_cpus': 12
	Fitting with cpus=12, gpus=0, mem=0.0/48.5 GB
Saving /content/california-house-prices/AutoGluonModels/models/RandomForestMSE/model.pkl
Saving /content/california-house-prices/AutoGluonModels/utils/attr/RandomForestMSE/y_pred_proba_val.pkl
	-361194.9404	 = Validation score   (-root_mean_squared_error)
	4816.53s	 = Training   runtime
	0.19s	 = Validation runtime
	13365.2	 = Inference  throughput (ro

*** Summary of fit() ***
Estimated performance of each model:
                 model      score_val              eval_metric  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2 -281822.375330  root_mean_squared_error       0.400613  4864.851567                0.000598           0.011277            2       True          4
1             LightGBM -327695.895205  root_mean_squared_error       0.093877    20.744567                0.093877          20.744567            1       True          2
2      RandomForestMSE -361194.940385  root_mean_squared_error       0.187053  4816.529906                0.187053        4816.529906            1       True          3
3           LightGBMXT -389418.266346  root_mean_squared_error       0.119085    27.565818                0.119085          27.565818            1       True          1
Number of models trained: 4
Types of models trained:
{'WeightedEnsembleModel', 'RFModel', 'LG

In [5]:
# test_identity = pd.read_csv(directory+'test_identity.csv')
# test_transaction = pd.read_csv(directory+'test_transaction.csv')
# test_data = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')  # same join applied to training files

y_predproba = predictor.predict(test_data)
y_predproba.head(5)  # some example predicted fraud-probabilities

Loading: /content/california-house-prices/AutoGluonModels/models/LightGBM/model.pkl
Loading: /content/california-house-prices/AutoGluonModels/models/LightGBMXT/model.pkl
Loading: /content/california-house-prices/AutoGluonModels/models/RandomForestMSE/model.pkl
Loading: /content/california-house-prices/AutoGluonModels/models/WeightedEnsemble_L2/model.pkl


,Sold Price
0,7.991716e+05
1,5.807125e+05
2,8.373142e+05
3,8.081618e+05
4,1.166623e+06


**dtype**: float32

When submitting predicted probabilities for classification competitions, it is imperative these correspond to the same class expected by Kaggle. For binary classification tasks, you can see which class AutoGluon's predicted probabilities correspond to via:

In [6]:
predictor.positive_class

For multiclass classification tasks, you can see which classes AutoGluon's predicted probabilities correspond to via:

In [7]:
predictor.class_labels  # classes in this list correspond to columns of predict_proba() output

Now, let's get prediction probabilities for the entire test data, while only getting the positive class predictions by specifying:

Now that we have made a prediction for each row in the test dataset, we can submit these predictions to Kaggle. Most Kaggle competitions provide a sample submission file, in which you can simply overwrite the sample predictions with your own as we do below:

In [8]:
submission = pd.read_csv(directory+'sample_submission.csv')
submission['Sold Price'] = y_predproba
submission.head()
submission.to_csv(directory+'my_submission.csv', index=False)

We have now completed steps (4)-(6) from the top of this tutorial. To submit your predictions to Kaggle, you can run the following command in your terminal (from the appropriate directory):

`kaggle competitions submit -c ieee-fraud-detection -f sample_submission.csv -m "my first submission"`

You can now play with different `fit()` arguments and feature-engineering techniques to try and maximize the rank of your submissions in the Kaggle Leaderboard!

**Tips to maximize predictive performance:**

*   Be sure to specify the appropriate evaluation metric if one is specified on the competition website! If you are unsure which metric is best, then simply do not specify this argument when invoking `fit()`; AutoGluon should still produce high-quality models by automatically inferring which metric to use.
*   If the training examples are time-based and the competition test examples come from future data, we recommend you reserve the most recently-collected training examples as a separate validation dataset passed to `fit()`. Otherwise, you do not need to specify a validation set yourself and AutoGluon will automatically partition the competition training data into its own training/validation sets.
* Beyond simply specifying `presets = 'best_quality'`, you may play with more advanced `fit()` arguments such as: `num_bag_folds`, `num_stack_levels`, `num_bag_sets`, `hyperparameter_tune_kwargs`, `hyperparameters`, `refit_full`. However we recommend spending most of your time on feature-engineering and just specify presets = `'best_quality'` inside the call to `fit()`.

**Troubleshooting:**


*   Check that you have the right user-permissions on your computer to access the data files downloaded from Kaggle.
*   For issues downloading Kaggle data or submitting predictions, check your Kaggle account setup and the Kaggle FAQ.

